1

In [1]:
import time
from PIL import Image # Pillow
import array

In [7]:
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate list"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output
def calc_pure_python(desired_width, max_iterations):
    """Create a list of zs and cs, build Julia set and display using PIL"""
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list and the initial condition.
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # 1000^2 grid with 300 iterations
    assert sum(output) == 33219980

if __name__ == "__main__":
    # Calculate the Julia set 
    # set draw_output
    calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 3.76291561126709 seconds


In [6]:
# area of complex space to be rendered
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193
def show_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert scale to [0...255]
    max_iterations = float(max(output_raw))
    print(max_iterations)
    scale_factor = float(max_iterations)
    scaled = [int(o / scale_factor * 255) for o in output_raw]
    output = array.array('B', scaled)  # array of unsigned ints
    # display
    im = Image.new("L", (width, width))
    # Greyscale
    im.frombytes(output.tobytes(), "raw", "L", 0, -1)
    im.show(title="Greyscale Julia Set")

def show_false_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert scale to [0...255]
    assert width * height == len(output_raw)
    # normalise to 0-255
    max_value = float(max(output_raw))
    output_raw_limited = [int(float(o) / max_value * 255) for o in output_raw]
    # fancy RGB
    output_rgb = (
        (o + (256 * o) + (256 ** 2) * o) * 16 for o in output_raw_limited)  # fancier
    # convert to bytes
    output_rgb = array.array('I', output_rgb)
    # display
    im = Image.new("RGB", (width, height))
    # False colour
    im.frombytes(output_rgb.tobytes(), "raw", "RGBX", 0, -1)
    im.show(title="False Greyscale Julia Set")

def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate list"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

def calc_pure_python( desired_width, max_iterations):
    """Create a list of zs and cs, build Julia set and display using PIL"""
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    width = len(x)
    height = len(y)
    # build a list of co-ordinates and the initial condition 
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # 1000^2 grid with 300 iterations
    assert sum(output) == 33219980
    

    
if __name__ == "__main__":
    # Calculate the Julia set
    calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 3.6870152950286865 seconds


2

In [4]:
import time
from functools import wraps
def timefn(function):
    @wraps(function)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = function(*args, **kwargs)
        t2 = time.time()
        total_time = t2 - t1
        print(f"{function.__name__} took {total_time} seconds")
        return result
    return measure_time
    
@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Executes Julia set calculation"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output
if __name__ == "__main__":
     # Compute Julia set with Python only; settings optimized for laptops
    calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 3.8623645305633545 seconds
calculate_z_serial_purepython took 3.862487316131592 seconds


3

In [79]:
%timeit -r 5 -n 10 calc_pure_python(draw_output=False,desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.530345916748047 seconds
calculate_z_serial_purepython took 6.530345916748047 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.49880051612854 seconds
calculate_z_serial_purepython took 6.49880051612854 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.498265027999878 seconds
calculate_z_serial_purepython took 6.498265027999878 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.521870851516724 seconds
calculate_z_serial_purepython took 6.521870851516724 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.5087502002716064 seconds
calculate_z_serial_purepython took 6.509751081466675 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.497542381286621 seconds
calculate_z_serial_purepython took 6.497542381286621 seconds
Length of x: 1000

4

In [9]:
import cProfile
import pstats
profiler = cProfile.Profile()
profiler.enable()
calc_pure_python( desired_width=1000, max_iterations=300)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumulative')
stats.print_stats()

Exception ignored When destroying _lsprof profiler:
Traceback (most recent call last):
  File "/tmp/ipykernel_6435/324011351.py", line 4, in <module>
RuntimeError: Cannot install a profile function while another profile function is being installed


Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.363462924957275 seconds
         36222204 function calls in 7.814 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    7.844    3.922 /home/juan/Desktop/tareas/tareas/.tareasvenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000    7.844    3.922 {built-in method builtins.exec}
        1    0.357    0.357    7.814    7.814 /tmp/ipykernel_6435/767614226.py:15(calc_pure_python)
        1    5.439    5.439    7.363    7.363 /tmp/ipykernel_6435/767614226.py:3(calculate_z_serial_purepython)
 34219980    1.925    0.000    1.925    0.000 {built-in method builtins.abs}
  2002000    0.090    0.000    0.090    0.000 {method 'append' of 'list' objects}
        1    0.003    0.003    0.003    0.003 {built-in method builtins.sum}
        3    0.000    0.000    0.000    0.000 {b

5

In [11]:
%load_ext snakeviz


cProfile.run('calc_pure_python(desired_width=1000, max_iterations=300)', 'profiling')

!snakeviz "profiling"

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.301686525344849 seconds
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fhome%2Fjuan%2FDesktop%2Ftareas%2Ftareas%2FDB4D%2Fpart%201%2Fprofiling
^C

Bye!


6

In [84]:
%load_ext line_profiler
%lprun -f calculate_z_serial_purepython calc_pure_python(desired_width=1000, max_iterations=300) 

c:\Users\alber\OneDrive\Escritorio\Files\adjdajad\.venv\lib\site-packages\line_profiler\ipython_extension.py:97: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding calculate_z_serial_purepython.__wrapped__ instead.
  profile = LineProfiler(*funcs)


Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 30.42072582244873 seconds
calculate_z_serial_purepython took 30.42172598838806 seconds


Timer unit: 1e-07 s

Total time: 30.4215 s
File: C:\Users\alber\AppData\Local\Temp\ipykernel_19648\243504150.py
Function: measure_time at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                               @wraps(function)
     3                                               def measure_time(*args, **kwargs):
     4         1         12.0     12.0      0.0          t1 = time.time()
     5         1  304213760.0    3e+08    100.0          result = function(*args, **kwargs)
     6         1         24.0     24.0      0.0          t2 = time.time()
     7         1          6.0      6.0      0.0          total_time = t2 - t1
     8         1       1656.0   1656.0      0.0          print(f"{function.__name__} took {total_time} seconds")
     9         1          2.0      2.0      0.0          return result

7

In [85]:
%reload_ext memory_profiler 
%memit calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.528759241104126 seconds
calculate_z_serial_purepython took 6.528759241104126 seconds
peak memory: 245.83 MiB, increment: 72.38 MiB
